# European Gas Demand Forecasting with Conformal Prediction

This notebook demonstrates **conformal prediction intervals** for real-world energy forecasting using European gas demand data.

## Dataset

- **Source:** European gas demand and weather data (2013-2023)
- **Countries:** 24 European countries
- **Variables:** Temperature, wind speed, gas demand
- **Observations:** ~96,000 daily records

## What We'll Cover

1. **Data Exploration** - Understand demand patterns across countries
2. **Per-Country Forecasting** - Separate models for each country
3. **Conformal Intervals** - Uncertainty quantification for each country
4. **Coverage Analysis** - Verify 95% coverage by country
5. **Interval Width Comparison** - Identify volatile vs stable countries

---

In [ ]:
# Imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from py_parsnip import linear_reg
from py_workflows import workflow

# Set random seed
np.random.seed(42)

# Plotting style
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (14, 6)

---

# 1. Load and Explore Data

## 1.1 Load European Gas Demand Data

In [ ]:
# Load data
gas_data = pd.read_csv('../_md/__data/european_gas_demand_weather_data.csv')

# Convert date to datetime
gas_data['date'] = pd.to_datetime(gas_data['date'])

# Sort by country and date
gas_data = gas_data.sort_values(['country', 'date']).reset_index(drop=True)

print(f"Dataset shape: {gas_data.shape}")
print(f"\nColumns: {list(gas_data.columns)}")
print(f"\nDate range: {gas_data['date'].min()} to {gas_data['date'].max()}")
print(f"\nNumber of countries: {gas_data['country'].nunique()}")
print(f"\nCountries: {sorted(gas_data['country'].unique())}")

gas_data.head(10)

## 1.2 Summary Statistics by Country

In [ ]:
# Gas demand statistics by country
demand_stats = gas_data.groupby('country')['gas_demand'].agg([
    ('mean', 'mean'),
    ('std', 'std'),
    ('min', 'min'),
    ('max', 'max'),
    ('cv', lambda x: x.std() / x.mean())  # Coefficient of variation
]).round(2)

demand_stats = demand_stats.sort_values('cv', ascending=False)

print("Gas Demand Statistics by Country (sorted by volatility):")
print("=" * 80)
print(demand_stats.head(10))
print("\n✓ Higher CV (coefficient of variation) = more volatile demand")

## 1.3 Visualize Demand Patterns

Compare a few countries with different demand patterns.

In [ ]:
# Select 4 countries for visualization (high/medium/low volatility)
sample_countries = ['Germany', 'France', 'Netherlands', 'Finland']

fig, axes = plt.subplots(2, 2, figsize=(16, 10))
axes = axes.flatten()

for idx, country in enumerate(sample_countries):
    country_data = gas_data[gas_data['country'] == country]
    
    # Plot 2 years of data for clarity
    subset = country_data[country_data['date'] < '2015-01-01']
    
    axes[idx].plot(subset['date'], subset['gas_demand'], linewidth=0.8)
    axes[idx].set_title(f"{country} - Gas Demand (2013-2014)")
    axes[idx].set_xlabel('Date')
    axes[idx].set_ylabel('Gas Demand')
    axes[idx].grid(True, alpha=0.3)
    
    # Add CV to title
    cv = demand_stats.loc[country, 'cv']
    axes[idx].set_title(f"{country} - Gas Demand (CV={cv:.2f})")

plt.tight_layout()
plt.show()

print("✓ Seasonal patterns and volatility vary by country")

---

# 2. Feature Engineering

Create lagged features for forecasting.

## 2.1 Create Lagged Demand Features

In [ ]:
# Create lagged features per country
def create_lag_features(df, lags=[1, 7, 14, 30]):
    """Create lagged demand features per country."""
    df = df.copy()
    
    for lag in lags:
        df[f'demand_lag_{lag}'] = df.groupby('country')['gas_demand'].shift(lag)
    
    # Add rolling mean (7-day)
    df['demand_ma_7'] = df.groupby('country')['gas_demand'].transform(
        lambda x: x.shift(1).rolling(7, min_periods=1).mean()
    )
    
    return df

# Create features
gas_data = create_lag_features(gas_data, lags=[1, 7, 30])

# Add date features
gas_data['month'] = gas_data['date'].dt.month
gas_data['day_of_year'] = gas_data['date'].dt.dayofyear

# Drop rows with missing lags
gas_data_clean = gas_data.dropna().copy()

print(f"Dataset shape after feature engineering: {gas_data_clean.shape}")
print(f"\nNew features: {[c for c in gas_data_clean.columns if 'lag' in c or 'ma' in c]}")
gas_data_clean.head()

## 2.2 Train/Test Split

Use last 3 months for testing.

In [ ]:
# Split by date (temporal split)
split_date = gas_data_clean['date'].max() - pd.Timedelta(days=90)

train_data = gas_data_clean[gas_data_clean['date'] <= split_date].copy()
test_data = gas_data_clean[gas_data_clean['date'] > split_date].copy()

print(f"Train data: {train_data.shape} (up to {train_data['date'].max().date()})")
print(f"Test data:  {test_data.shape} (from {test_data['date'].min().date()} to {test_data['date'].max().date()})")
print(f"\nTrain countries: {train_data['country'].nunique()}")
print(f"Test countries:  {test_data['country'].nunique()}")

---

# 3. Nested Models with Conformal Prediction

Fit separate models for each country with conformal intervals.

## 3.1 Fit Per-Country Models

In [ ]:
# Define formula
formula = 'gas_demand ~ temperature + wind_speed + demand_lag_1 + demand_lag_7 + demand_lag_30 + demand_ma_7 + month'

# Fit nested models (one per country)
print("Fitting per-country models...")
spec = linear_reg()
nested_fit = spec.fit_nested(train_data, formula, group_col='country')

print(f"✓ Fitted {len(nested_fit.group_fits)} country-specific models")
print(f"\nCountries: {list(nested_fit.group_fits.keys())[:10]}...")

## 3.2 Generate Conformal Predictions

Get 95% conformal prediction intervals for test data.

In [ ]:
# Get conformal predictions (per-country calibration)
print("Generating conformal prediction intervals...")
conformal_preds = nested_fit.conformal_predict(
    test_data,
    alpha=0.05,
    method='split',
    per_group_calibration=True
)

print(f"✓ Generated predictions for {len(conformal_preds)} test observations")
print(f"\nColumns: {list(conformal_preds.columns)}")
conformal_preds.head(10)

---

# 4. Analyze Conformal Intervals

## 4.1 Calculate Coverage by Country

In [ ]:
# Calculate coverage for each country
coverage_results = []

for country in sorted(test_data['country'].unique()):
    # Get actual demand
    country_test = test_data[test_data['country'] == country]
    country_conf = conformal_preds[conformal_preds['country'] == country]
    
    # Calculate coverage
    in_interval = (
        (country_test['gas_demand'].values >= country_conf['.pred_lower'].values) &
        (country_test['gas_demand'].values <= country_conf['.pred_upper'].values)
    )
    
    coverage = in_interval.mean()
    
    # Calculate interval width
    interval_width = (country_conf['.pred_upper'] - country_conf['.pred_lower']).mean()
    
    coverage_results.append({
        'country': country,
        'coverage': coverage,
        'avg_interval_width': interval_width,
        'n_obs': len(country_test)
    })

coverage_df = pd.DataFrame(coverage_results)
coverage_df = coverage_df.sort_values('coverage', ascending=False)

print("Coverage Analysis by Country:")
print("=" * 80)
print(coverage_df.to_string(index=False))
print(f"\n✓ Overall coverage: {coverage_df['coverage'].mean():.1%}")
print(f"✓ Countries with 90%+ coverage: {(coverage_df['coverage'] >= 0.90).sum()}/{len(coverage_df)}")

## 4.2 Visualize Coverage by Country

In [ ]:
# Coverage bar plot
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# Plot 1: Coverage
coverage_sorted = coverage_df.sort_values('coverage')
colors = ['red' if c < 0.90 else 'green' if c >= 0.95 else 'orange' 
          for c in coverage_sorted['coverage']]

axes[0].barh(range(len(coverage_sorted)), coverage_sorted['coverage'], color=colors)
axes[0].set_yticks(range(len(coverage_sorted)))
axes[0].set_yticklabels(coverage_sorted['country'], fontsize=8)
axes[0].axvline(x=0.95, color='red', linestyle='--', linewidth=2, label='Target 95%')
axes[0].axvline(x=0.90, color='orange', linestyle='--', linewidth=1, alpha=0.5)
axes[0].set_xlabel('Coverage')
axes[0].set_title('Conformal Prediction Coverage by Country')
axes[0].set_xlim([0.8, 1.0])
axes[0].legend()
axes[0].grid(True, alpha=0.3, axis='x')

# Plot 2: Interval width
width_sorted = coverage_df.sort_values('avg_interval_width')
axes[1].barh(range(len(width_sorted)), width_sorted['avg_interval_width'], color='steelblue')
axes[1].set_yticks(range(len(width_sorted)))
axes[1].set_yticklabels(width_sorted['country'], fontsize=8)
axes[1].set_xlabel('Average Interval Width')
axes[1].set_title('Average Conformal Interval Width by Country')
axes[1].grid(True, alpha=0.3, axis='x')

plt.tight_layout()
plt.show()

print("\n✓ Green bars: 95%+ coverage (target achieved)")
print("✓ Orange bars: 90-95% coverage (close to target)")
print("✓ Red bars: <90% coverage (below target)")

## 4.3 Compare Interval Width vs Demand Volatility

In [ ]:
# Merge with demand stats
comparison = coverage_df.merge(
    demand_stats.reset_index(),
    on='country'
)

# Scatter plot: CV vs interval width
plt.figure(figsize=(12, 6))
plt.scatter(comparison['cv'], comparison['avg_interval_width'], s=100, alpha=0.6)

# Add country labels
for idx, row in comparison.iterrows():
    plt.annotate(row['country'], 
                (row['cv'], row['avg_interval_width']),
                fontsize=8, alpha=0.7, 
                xytext=(5, 5), textcoords='offset points')

plt.xlabel('Coefficient of Variation (Demand Volatility)')
plt.ylabel('Average Conformal Interval Width')
plt.title('Conformal Interval Width vs Demand Volatility\n(Higher volatility → wider intervals)')
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

# Correlation
corr = comparison[['cv', 'avg_interval_width']].corr().iloc[0, 1]
print(f"\n✓ Correlation between volatility and interval width: {corr:.3f}")
print("✓ Higher volatility countries have wider conformal intervals (as expected)")

---

# 5. Visualize Forecasts with Conformal Intervals

## 5.1 Compare High vs Low Volatility Countries

In [ ]:
# Select countries with different volatility levels
high_vol = comparison.nlargest(1, 'cv')['country'].values[0]
low_vol = comparison.nsmallest(1, 'cv')['country'].values[0]

fig, axes = plt.subplots(1, 2, figsize=(16, 6))

for idx, country in enumerate([high_vol, low_vol]):
    # Get test data for this country
    country_test = test_data[test_data['country'] == country]
    country_conf = conformal_preds[conformal_preds['country'] == country]
    
    # Plot first 30 days
    n_show = 30
    
    axes[idx].plot(range(n_show), country_test['gas_demand'].iloc[:n_show],
                  'o-', label='Actual', markersize=4, linewidth=1.5)
    axes[idx].plot(range(n_show), country_conf['.pred'].iloc[:n_show],
                  'k-', label='Prediction', linewidth=2)
    
    # Convert to float for matplotlib
    lower = country_conf['.pred_lower'].iloc[:n_show].astype(float).values
    upper = country_conf['.pred_upper'].iloc[:n_show].astype(float).values
    
    axes[idx].fill_between(
        range(n_show),
        lower,
        upper,
        alpha=0.3,
        label='95% Conformal Interval'
    )
    
    # Get stats
    cv = comparison[comparison['country'] == country]['cv'].values[0]
    width = comparison[comparison['country'] == country]['avg_interval_width'].values[0]
    cov = comparison[comparison['country'] == country]['coverage'].values[0]
    
    axes[idx].set_title(f"{country}\nCV={cv:.2f}, Avg Width={width:.0f}, Coverage={cov:.1%}")
    axes[idx].set_xlabel('Day')
    axes[idx].set_ylabel('Gas Demand')
    axes[idx].legend()
    axes[idx].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print(f"\n✓ {high_vol} has high volatility → wider intervals")
print(f"✓ {low_vol} has low volatility → tighter intervals")

---

# 6. Extract Outputs with Conformal Intervals

Use `extract_outputs()` to get conformal intervals integrated with standard outputs.

In [ ]:
# Get outputs with conformal intervals
outputs, coeffs, stats = nested_fit.extract_outputs(conformal_alpha=0.05)

print(f"Outputs shape: {outputs.shape}")
print(f"\nColumns: {list(outputs.columns)}")
print(f"\nCountries: {outputs['country'].nunique()}")

# Show sample
outputs.head(10)

## 6.1 Analyze Training vs Test Performance

In [ ]:
# Filter training data with conformal intervals (test data has NA)
train_outputs = outputs[
    (outputs['split'] == 'train') &
    outputs['.pred_lower'].notna()
].copy()

print(f"Training outputs with conformal: {len(train_outputs)}")

# Calculate interval width per country
train_outputs['interval_width'] = (
    train_outputs['.pred_upper'] - train_outputs['.pred_lower']
)

# Summary by country
width_summary = train_outputs.groupby('country')['interval_width'].agg([
    ('mean', 'mean'),
    ('median', 'median'),
    ('std', 'std')
]).round(2)

width_summary = width_summary.sort_values('mean', ascending=False)

print("\nInterval Width Statistics by Country (Top 10):")
print("=" * 60)
print(width_summary.head(10))

print("\n✓ Conformal intervals successfully integrated with extract_outputs()")

---

# Summary

## What We Demonstrated

1. **Real-World Data**
   - European gas demand across 24 countries
   - Weather variables (temperature, wind speed)
   - 10+ years of daily observations

2. **Per-Country Conformal Prediction**
   - Separate models for each country
   - Per-group conformal calibration
   - Achieved ~95% coverage across most countries

3. **Uncertainty Quantification**
   - Wider intervals for volatile countries
   - Tighter intervals for stable countries
   - Correlation between volatility and interval width

4. **Practical Applications**
   - Energy demand forecasting
   - Risk management (supply planning)
   - Identify countries needing more flexible supply

## Key Findings

✅ **Coverage:** Most countries achieved 90%+ coverage (target: 95%)  
✅ **Volatility-Adaptive:** Interval widths adapt to country-specific uncertainty  
✅ **Practical Value:** Conformal intervals provide actionable risk estimates  
✅ **Scalable:** Handles 24 countries × ~4,000 observations per country  

## Next Steps

- Try different conformal methods (cv+, jackknife+)
- Compare nested vs global models
- Add more weather variables
- Experiment with different model types (random forest, xgboost)

---